<a href="https://www.kaggle.com/code/abhichaudhary00/car-price-randomforest?scriptVersionId=180809280" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import pandas as pd
import seaborn as sn
from datetime import date
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score , mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')


In [2]:
df = pd.read_csv("/kaggle/input/car-price-prediction-challenge/car_price_prediction.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19237 entries, 0 to 19236
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ID                19237 non-null  int64  
 1   Price             19237 non-null  int64  
 2   Levy              19237 non-null  object 
 3   Manufacturer      19237 non-null  object 
 4   Model             19237 non-null  object 
 5   Prod. year        19237 non-null  int64  
 6   Category          19237 non-null  object 
 7   Leather interior  19237 non-null  object 
 8   Fuel type         19237 non-null  object 
 9   Engine volume     19237 non-null  object 
 10  Mileage           19237 non-null  object 
 11  Cylinders         19237 non-null  float64
 12  Gear box type     19237 non-null  object 
 13  Drive wheels      19237 non-null  object 
 14  Doors             19237 non-null  object 
 15  Wheel             19237 non-null  object 
 16  Color             19237 non-null  object

In [4]:
df.head()

,ID,Price,Levy,Manufacturer,Model,Prod. year,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Doors,Wheel,Color,Airbags
0,45654403,13328,1399,LEXUS,RX 450,2010,Jeep,Yes,Hybrid,3.5,186005 km,6.0,Automatic,4x4,04-May,Left wheel,Silver,12
1,44731507,16621,1018,CHEVROLET,Equinox,2011,Jeep,No,Petrol,3,192000 km,6.0,Tiptronic,4x4,04-May,Left wheel,Black,8
2,45774419,8467,-,HONDA,FIT,2006,Hatchback,No,Petrol,1.3,200000 km,4.0,Variator,Front,04-May,Right-hand drive,Black,2
3,45769185,3607,862,FORD,Escape,2011,Jeep,Yes,Hybrid,2.5,168966 km,4.0,Automatic,4x4,04-May,Left wheel,White,0
4,45809263,11726,446,HONDA,FIT,2014,Hatchback,Yes,Petrol,1.3,91901 km,4.0,Automatic,Front,04-May,Left wheel,Silver,4


In [5]:
df.shape

(19237, 18)

In [6]:
df.duplicated().sum()

313

In [7]:
df.drop_duplicates(inplace = True)

In [8]:
df.nunique()

ID                  18924
Price                2315
Levy                  559
Manufacturer           65
Model                1590
Prod. year             54
Category               11
Leather interior        2
Fuel type               7
Engine volume         107
Mileage              7687
Cylinders              13
Gear box type           4
Drive wheels            3
Doors                   3
Wheel                   2
Color                  16
Airbags                17
dtype: int64

In [9]:
year = date.today().year
df['car_age'] = year - df['Prod. year']

In [10]:

df['Wheel'].unique()

array(['Left wheel', 'Right-hand drive'], dtype=object)

In [11]:
df['Wheel'].replace({'Left wheel': "Left-hand drive"} , inplace = True)

In [12]:
df['Manufacturer'].unique()

array(['LEXUS', 'CHEVROLET', 'HONDA', 'FORD', 'HYUNDAI', 'TOYOTA',
       'MERCEDES-BENZ', 'OPEL', 'PORSCHE', 'BMW', 'JEEP', 'VOLKSWAGEN',
       'AUDI', 'RENAULT', 'NISSAN', 'SUBARU', 'DAEWOO', 'KIA',
       'MITSUBISHI', 'SSANGYONG', 'MAZDA', 'GMC', 'FIAT', 'INFINITI',
       'ALFA ROMEO', 'SUZUKI', 'ACURA', 'LINCOLN', 'VAZ', 'GAZ',
       'CITROEN', 'LAND ROVER', 'MINI', 'DODGE', 'CHRYSLER', 'JAGUAR',
       'ISUZU', 'SKODA', 'DAIHATSU', 'BUICK', 'TESLA', 'CADILLAC',
       'PEUGEOT', 'BENTLEY', 'VOLVO', 'სხვა', 'HAVAL', 'HUMMER', 'SCION',
       'UAZ', 'MERCURY', 'ZAZ', 'ROVER', 'SEAT', 'LANCIA', 'MOSKVICH',
       'MASERATI', 'FERRARI', 'SAAB', 'LAMBORGHINI', 'ROLLS-ROYCE',
       'PONTIAC', 'SATURN', 'ASTON MARTIN', 'GREATWALL'], dtype=object)

In [13]:
M_count = df["Manufacturer"].value_counts()
for Manufacturer , count in M_count.items():
    print(f"{Manufacturer} : {count}")

HYUNDAI : 3729
TOYOTA : 3606
MERCEDES-BENZ : 2043
FORD : 1088
CHEVROLET : 1047
BMW : 1036
HONDA : 960
LEXUS : 927
NISSAN : 645
VOLKSWAGEN : 571
SSANGYONG : 439
KIA : 417
OPEL : 396
MITSUBISHI : 288
SUBARU : 274
AUDI : 254
MAZDA : 175
JEEP : 138
DAEWOO : 91
DODGE : 90
SUZUKI : 76
FIAT : 76
PORSCHE : 52
LAND ROVER : 48
VAZ : 47
MINI : 47
JAGUAR : 42
RENAULT : 36
INFINITI : 26
CHRYSLER : 26
SKODA : 20
VOLVO : 19
PEUGEOT : 17
BUICK : 16
LINCOLN : 15
ACURA : 15
GMC : 15
CADILLAC : 14
DAIHATSU : 13
GAZ : 12
UAZ : 12
CITROEN : 9
SCION : 7
HUMMER : 5
MASERATI : 4
MERCURY : 4
ISUZU : 4
ALFA ROMEO : 4
MOSKVICH : 4
ROVER : 3
FERRARI : 2
SAAB : 2
ROLLS-ROYCE : 2
სხვა : 2
SEAT : 2
ZAZ : 2
BENTLEY : 2
LANCIA : 1
HAVAL : 1
LAMBORGHINI : 1
TESLA : 1
PONTIAC : 1
SATURN : 1
ASTON MARTIN : 1
GREATWALL : 1


In [14]:
# df["Price"].describe()
df.describe()

,ID,Price,Prod. year,Cylinders,Airbags,car_age
count,1.892400e+04,1.892400e+04,18924.000000,18924.000000,18924.000000,18924.000000
mean,4.557538e+07,1.858744e+04,2010.914236,4.580216,6.568379,13.085764
std,9.375468e+05,1.921356e+05,5.665749,1.200223,4.322323,5.665749
min,2.074688e+07,1.000000e+00,1939.000000,1.000000,0.000000,4.000000
25%,4.569501e+07,5.331000e+03,2009.000000,4.000000,4.000000,9.000000
50%,4.577191e+07,1.317200e+04,2012.000000,4.000000,6.000000,12.000000
75%,4.580174e+07,2.206300e+04,2015.000000,4.000000,12.000000,15.000000
max,4.581665e+07,2.630750e+07,2020.000000,16.000000,16.000000,85.000000


In [15]:
df = df.drop(["ID" , "Doors"] , axis=1)

In [16]:
df.head(2)

,Price,Levy,Manufacturer,Model,Prod. year,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Wheel,Color,Airbags,car_age
0,13328,1399,LEXUS,RX 450,2010,Jeep,Yes,Hybrid,3.5,186005 km,6.0,Automatic,4x4,Left-hand drive,Silver,12,14
1,16621,1018,CHEVROLET,Equinox,2011,Jeep,No,Petrol,3,192000 km,6.0,Tiptronic,4x4,Left-hand drive,Black,8,13


In [17]:
df["Levy"].replace({ "-" : 0 } , inplace = True)
df['Levy'] = df['Levy'].astype('int')

In [18]:
df['Mileage'] = df['Mileage'].astype('str')
df['Mileage'] = df['Mileage'].str.replace('km' , "")
df['Mileage'] = df['Mileage'].astype("int")

In [19]:
df

,Price,Levy,Manufacturer,Model,Prod. year,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Wheel,Color,Airbags,car_age
0,13328,1399,LEXUS,RX 450,2010,Jeep,Yes,Hybrid,3.5,186005,6.0,Automatic,4x4,Left-hand drive,Silver,12,14
1,16621,1018,CHEVROLET,Equinox,2011,Jeep,No,Petrol,3,192000,6.0,Tiptronic,4x4,Left-hand drive,Black,8,13
2,8467,0,HONDA,FIT,2006,Hatchback,No,Petrol,1.3,200000,4.0,Variator,Front,Right-hand drive,Black,2,18
3,3607,862,FORD,Escape,2011,Jeep,Yes,Hybrid,2.5,168966,4.0,Automatic,4x4,Left-hand drive,White,0,13
4,11726,446,HONDA,FIT,2014,Hatchback,Yes,Petrol,1.3,91901,4.0,Automatic,Front,Left-hand drive,Silver,4,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19232,8467,0,MERCEDES-BENZ,CLK 200,1999,Coupe,Yes,CNG,2.0 Turbo,300000,4.0,Manual,Rear,Left-hand drive,Silver,5,25
19233,15681,831,HYUNDAI,Sonata,2011,Sedan,Yes,Petrol,2.4,161600,4.0,Tiptronic,Front,Left-hand drive,Red,8,13
19234,26108,836,HYUNDAI,Tucson,2010,Jeep,Yes,Diesel,2,116365,4.0,Automatic,Front,Left-hand drive,Grey,4,14
19235,5331,1288,CHEVROLET,Captiva,2007,Jeep,Yes,Diesel,2,51258,4.0,Automatic,Front,Left-hand drive,Black,4,17


In [20]:
data_numeric = df.select_dtypes(exclude='object')

for col in data_numeric.columns:
    q1 = data_numeric[col].quantile(0.25)
    q3 = data_numeric[col].quantile(0.75)
    iqr = q3 - q1
    low = q1 - (1.5 * iqr)
    high = q3 + (1.5 * iqr)
    outliers = ((data_numeric[col] > high) | (data_numeric[col] < low)).sum()
    total = data_numeric.shape[0]
    print(f"Total outliers in {col} are: {outliers} - {round(100 * (outliers / total), 2)}%")
    
    if outliers > 0:
        df = df.loc[(df[col] <= high) & (df[col] >= low)]


Total outliers in Price are: 1055 - 5.57%
Total outliers in Levy are: 160 - 0.85%
Total outliers in Prod. year are: 962 - 5.08%
Total outliers in Mileage are: 635 - 3.36%
Total outliers in Cylinders are: 4765 - 25.18%
Total outliers in Airbags are: 0 - 0.0%
Total outliers in car_age are: 962 - 5.08%


In [21]:
dobject = df.select_dtypes(include = 'object')
dnumeric = df.select_dtypes(exclude = 'object')

In [22]:
la = LabelEncoder()

In [23]:
label_encoders = {}
for col in dobject.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    dobject[col] = le.fit_transform(dobject[col])
    label_encoders[col] = le

In [24]:
for i in range(0, dobject.shape[1]):
    dobject.iloc[:, i] = la.fit_transform(dobject.iloc[:, i])

In [25]:
df = pd.concat([dobject, dnumeric], axis=1)

In [26]:
x = df.drop('Price' , axis = 1)
y = df['Price']

In [27]:
x_train, x_test, y_train,y_test = train_test_split(x, y,test_size = 0.2 , random_state=92)

In [28]:
model = RandomForestRegressor(n_estimators = 48 , random_state= 72)

In [29]:
model.fit(x_train , y_train)

RandomForestRegressor(n_estimators=48, random_state=72)

In [30]:
prediction = model.predict(x_test)

In [31]:
score = model.score (x_test,y_test)

In [32]:
prediction

array([15890.6875    , 15681.        , 14105.1875    , ...,
       28666.02083333, 24366.20833333, 33598.39583333])

In [33]:
score*100

78.07422499798177